In [1]:
#版本说明

#本程序终于来到5.0版本，这个版本的特点是
#1.输入地图参数，勇士即可到达终点
#2.在寻找到金子之前，勇士不会在离开游戏
#3.没有设计杀怪兽的机制，勇士会躲着怪兽和洞跑

In [2]:
#这个cell定义class map，可以输入怪物和洞的信息，从而判断出哪里有风哪里有气味，哪里有黄金， 保存为坐标的形式
#所有坐标都是行+列的形式
import numpy as np
class map:
    def __init__(self,monster1=[],monster2=[],monster3=[],hole1=[],hole2=[],hole3=[],gold=[],wind=[],smell=[]):
        self.monster1=monster1
        self.monster2=monster2
        self.monster3=monster3
        self.hole1=hole1
        self.hole2=hole2
        self.hole3=hole3
        self.gold=gold
        
    def generate(self):
    #用来输入怪物，洞，黄金的位置
    
        a=[0,0]
        a[0]=input('请输入怪物1的行')
        a[1]=input('请输入怪物1的列')
        b=[0,0]
        b[0]=input('请输入怪物2的行')
        b[1]=input('请输入怪物2的列')
        c=[0,0]
        c[0]=input('请输入怪物3的行')
        c[1]=input('请输入怪物3的列')
        d=[0,0]
        d[0]=input('请输入洞1的行')
        d[1]=input('请输入洞1的列')
        e=[0,0]
        e[0]=input('请输入洞2的行')
        e[1]=input('请输入洞2的列')
        f=[0,0]
        f[0]=input('请输入洞3的行')
        f[1]=input('请输入洞3的列')
        g=[0,0]
        g[0]=input('请输入黄金的行')
        g[1]=input('请输入黄金的列')


        
        self.monster1=[int(a[0]),int(a[1])]
        self.monster2=[int(b[0]),int(b[1])]
        self.monster3=[int(c[0]),int(c[1])]
        self.hole1=[int(d[0]),int(d[1])]
        self.hole2=[int(e[0]),int(e[1])]
        self.hole3=[int(f[0]),int(f[1])]
        self.gold=[int(g[0]),int(g[1])]
        return
    
    def clarify(self):
    #判断气味和风的坐标，并且清除重复的坐标
        self.wind=[[self.hole1[0]-1,self.hole1[1]],[self.hole1[0]+1,self.hole1[1]],[self.hole1[0],self.hole1[1]-1],[self.hole1[0],self.hole1[1]+1],
                   [self.hole2[0]-1,self.hole2[1]],[self.hole2[0]+1,self.hole2[1]],[self.hole2[0],self.hole2[1]-1],[self.hole2[0],self.hole2[1]+1],
                   [self.hole3[0]-1,self.hole3[1]],[self.hole3[0]+1,self.hole3[1]],[self.hole3[0],self.hole3[1]-1],[self.hole3[0],self.hole3[1]+1]]
        wind=[]
        for i in self.wind:
            if i not in wind:
                wind.append(i)
        self.wind=wind
        self.smell=[[self.monster1[0]-1,self.monster1[1]],[self.monster1[0]+1,self.monster1[1]],[self.monster1[0],self.monster1[1]-1],[self.monster1[0],self.monster1[1]+1],
                   [self.monster2[0]-1,self.monster2[1]],[self.monster2[0]+1,self.monster2[1]],[self.monster2[0],self.monster2[1]-1],[self.monster2[0],self.monster2[1]+1],
                   [self.monster3[0]-1,self.monster3[1]],[self.monster3[0]+1,self.monster3[1]],[self.monster3[0],self.monster3[1]-1],[self.monster3[0],self.monster3[1]+1]]
        smell=[]
        for i in self.smell:
            if i not in smell:
                smell.append(i)
        self.smell=smell



In [3]:
#这个cell定义sight 视野， 用来描述勇士目前知道的哪里有可能有怪兽，哪里有可能有洞（Pmonster，Phole），
#并且描述哪里没有怪兽，哪里没有洞（Nmonster，Nhole），
#clear方法用来删除上面四个list里面的重复项

Nmonster=[]
Nhole=[]
gold=[]
Pmonster=[]
Phole=[]

#Pmonster表示可能有怪物
#Nmonster表示一定没怪物
        
def clear(a,b,c,d):
#is used to delete repetitive unit
    global Nmonster
    global Nhole
    global Pmonster
    global Phole
    new1=[]
    for i in a:
        if i not in new1:
            new1.append(i)
    Nmonster=new1
        
    new2=[]
    for i in b:
        if i not in new2:
            new2.append(i)
    Nhole=new2       
        
    new3=[]
    for i in c:
        if i not in new3:
            new3.append(i)
    Pmonster=new3
        
    new4=[]
    for i in d:
        if i not in new4:
            new4.append(i)
    Phole=new4


In [4]:
#这个cell用来描述勇士的状态，输入map，输入位置loc，输入will表示向某个位置走的意愿大小（离开原点越远，意愿越强烈，去过某个房间次数越多，意愿越小）
class state:
    def __init__(self,m,loc=[1,1],parent=None,will=0):
        
    #m is map

        self.m=m
        self.loc=loc
        self.parent=parent
        self.will=will
        #用离开原点的距离来判断前进意愿
    
    def ifwind(self,m):
            if self.loc in m.wind:
                return True
            else:
                return False
        
        
    def ifsmell(self,m):
            if self.loc in m.smell:
                return True
            else:
                return False
        
    def ifgold(self,m):
            if self.loc in m.gold:
                return True
            else:
                return False
    


    def wheredanger(self,m,loc):
        global Nmonster
        global Nhole
        global Pmonster
        global Phole
    #这个函数用来判断危险，方法是，如果发现本格子有气味（风），那么周围四个格子都可能有怪兽（洞），加入Pmonster和Phole列表
    #如果发现本格子没有气味也没有风，那么周围四个格子一定没有怪兽或者洞，加入Nmonster和Nhole列表

    #在Pmonster和Phole列表中删除Nmonster和Nhole中有的元素

            
        _wind=self.ifwind(m)
        _smell=self.ifsmell(m)
        _gold=self.ifgold(m)

    
                    
        
        if _smell:
            if self.loc[0]>=2:
                Pmonster.append([loc[0]-1,loc[1]])
            if self.loc[0]<=9:
                Pmonster.append([loc[0]+1,loc[1]])
            if self.loc[1]>=2:
                Pmonster.append([loc[0],loc[1]-1])
            if self.loc[1]<=9:
                Pmonster.append([loc[0],loc[1]+1])
            
        else:
            if self.loc[0]>=2:
                Nmonster.append([loc[0]-1,loc[1]])
            if self.loc[0]<=9:
                Nmonster.append([loc[0]+1,loc[1]])
            if self.loc[1]>=2:
                Nmonster.append([loc[0],loc[1]-1])
            if self.loc[1]<=9:
                Nmonster.append([loc[0],loc[1]+1])
            
        if _wind:
            if self.loc[0]>=2:
                Phole.append([loc[0]-1,loc[1]])
            if self.loc[0]<=9:
                Phole.append([loc[0]+1,loc[1]])
            if self.loc[1]>=2:
                Phole.append([loc[0],loc[1]-1])
            if self.loc[1]<=9:
                Phole.append([loc[0],loc[1]+1])
            
        else:
            if self.loc[0]>=2:
                Nhole.append([loc[0]-1,loc[1]])
            if self.loc[0]<=9:
                Nhole.append([loc[0]+1,loc[1]])
            if self.loc[1]>=2:
                Nhole.append([loc[0],loc[1]-1])
            if self.loc[1]<=9:
                Nhole.append([loc[0],loc[1]+1])
                
        clear(Nmonster,Nhole,Pmonster,Phole)
        #进行清理
        

        #从有可能有怪物的里面删除确定没有怪物的单元
        #我们认为10，10一定是安全的
        Nmonster.append([loc[0],loc[1]])
        Nhole.append([loc[0],loc[1]])
        if [10,10] in Pmonster:
            Pmonster.remove([10,10])
        if [10,10] in Phole:
            Phole.remove([10,10])
 
        for i1 in Pmonster:
            for j1 in Nmonster:
                if i1==j1:
                    Pmonster.remove(i1)
        

        for i2 in Nhole:
            for j2 in Phole:
                if i2==j2:
                    Phole.remove(j2)
        
        clear(Nmonster,Nhole,Pmonster,Phole)
    
    def generatesubstate(self,m,loc):
    #这个函数是为了生成下一步可以走的state：只要这个state不认为可能存在怪兽或者可能存在洞，就可以走
    
        subStates = []
        self.wheredanger(m,loc)
        #先进行一次观察，看看哪里危险
        
        #看看能不能向左走
        if loc[1]>=2 and  ([loc[0],loc[1]-1] not in Pmonster) and ([loc[0],loc[1]-1] not in Phole):

                new1=state(m,loc=[loc[0],loc[1]-1])
                
                if self.m.gold in path:
                    new1.will =-5*path.count([loc[0],loc[1]-1])+loc[0]+loc[1]-1
                else:
                    new1.will=-5*path.count([loc[0],loc[1]-1])
                    
                subStates.append(new1)
        #看看能不能向右走
        if loc[1]<=9 and  ([loc[0],loc[1]+1] not in Pmonster) and ([loc[0],loc[1]+1] not in Phole):

                new2=state(m,loc=[loc[0],loc[1]+1])
                
                
                if self.m.gold in path:
                    new2.will =-5*path.count([loc[0],loc[1]+1])+loc[0]+loc[1]+1
                else:
                    new2.will=-5*path.count([loc[0],loc[1]+1])
                    
                subStates.append(new2)        
        #看看能不能向上走
        if loc[0]<=9 and  ([loc[0]+1,loc[1]] not in Pmonster) and ([loc[0]+1,loc[1]] not in Phole):
                new3=state(m,loc=[loc[0]+1,loc[1]])
                if self.m.gold in path:
                    new3.will =-5*path.count([loc[0]+1,loc[1]])+loc[0]+loc[1]+1
                else:
                    new3.will=-5*path.count([loc[0]+1,loc[1]])
                subStates.append(new3)
        #看看能不能向下走
        if loc[0]>=2 and  ([loc[0]-1,loc[1]] not in Pmonster) and ([loc[0]-1,loc[1]] not in Phole):

                new4=state(m,loc=[loc[0]-1,loc[1]])
                if self.m.gold in path:
                    new4.will =-5*path.count([loc[0]-1,loc[1]])+loc[0]+loc[1]-1
                else:
                    new4.will=-5*path.count([loc[0]-1,loc[1]])
                subStates.append(new4)
        return subStates
        #对will进行修改:不太愿意去已经去过的房间，尽量去可能多的房间。且当找到金子之后，会迅速离开
        

    def solve(self,m):
    #这个函数用来解决问题，采用的方式是搜索的方式，启发函数用到了will的大小：优先走will比较大的情况
        openTable=[]
        closeTable=[]
        openTable.append(self)
        steps=0
       
        global path
        path=[]
        while len(openTable)>0:
            n=openTable.pop(0)
            path.append(n.loc)
      
            closeTable.append(n)
            n.wheredanger(m,loc=n.loc)
            substates=n.generatesubstate(m,loc=n.loc)

            openTable=substates


            openTable.sort(key=lambda x:x.will,reverse=True)
          
                #sort the opentable in terms of the will
            steps=steps+1
            
            if n.loc[0]==10 and n.loc[1]==10 and self.m.gold in path:
                break



            

        return path,steps

In [5]:
#this cell is used to solve

#生成地图
map1=map()
map1.generate()
map1.clarify()


#生成勇士
warrior=state(m=map1)

#解决问题
[a,b]=warrior.solve(m=map1)

#计算得分
if map1.gold in path:
    point=1000-b
else:
    point=-b

#打印结果
print('path: ',a)
print('steps: ',b)
print('point: ',point)





请输入怪物1的行3
请输入怪物1的列3
请输入怪物2的行8
请输入怪物2的列3
请输入怪物3的行3
请输入怪物3的列8
请输入洞1的行5
请输入洞1的列5
请输入洞2的行2
请输入洞2的列6
请输入洞3的行9
请输入洞3的列9
请输入黄金的行5
请输入黄金的列6
path:  [[1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 5], [2, 5], [2, 4], [2, 3], [2, 2], [2, 1], [3, 1], [3, 2], [2, 2], [2, 1], [3, 1], [4, 1], [4, 2], [4, 3], [4, 2], [5, 2], [5, 1], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 4], [7, 4], [7, 3], [7, 2], [7, 1], [8, 1], [8, 2], [7, 2], [7, 1], [8, 1], [9, 1], [9, 2], [9, 3], [9, 2], [10, 2], [10, 1], [10, 2], [10, 3], [10, 4], [10, 5], [10, 6], [10, 7], [10, 8], [10, 9], [10, 10], [9, 10], [10, 10], [10, 9], [10, 8], [9, 8], [9, 7], [9, 6], [9, 5], [9, 4], [8, 4], [8, 5], [8, 6], [8, 7], [8, 8], [8, 9], [8, 8], [7, 8], [7, 7], [7, 6], [7, 5], [7, 4], [7, 3], [7, 2], [8, 2], [8, 1], [9, 1], [10, 1], [10, 2], [10, 3], [10, 4], [10, 5], [10, 6], [10, 7], [9, 7], [9, 6], [9, 5], [9, 4], [9, 3], [9, 2], [9, 1], [10, 1], [10, 2], [10, 3], [10, 4], [10, 5], [10, 6], [10, 7], [10, 8], [9, 8], [9, 7], [8, 